While working with data pytorch has two primitives
1. torch.utils.data.DataLoader(Wraps an iterable around the dataset)
2. torch.utils.data.Dataset(Stores samples and their labels)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as **TorchText, TorchVision, and TorchAudio**, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.

`torchvision.datasets` module contains Dataset Object for many real world vision data, we will be using fashion MNIST
- Every torchvision dataset has two arguments **transform** and **target_transform** to modify samples and targets

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor(),
)

In [3]:
testing_data = datasets.FashionMNIST(root = 'data',train = False, download = False, transform = ToTensor())

training and testing_data or instances of Dataset class which will be passed to DataLoader to wrap an iterable around our dataset

why do we wrap an iterable?
  - it supports automatic batching, sampling, shuffling, multiprocess data loading
 
here we will keep the batch size of 64, so each element of Dataloader iterable will return a batch of 64

In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size = batch_size)
test_dataloader = DataLoader(testing_data, batch_size = batch_size)

# lets take a moment to understand how our data is divided after forming the batches
# firstly we had 60k images, each of size 28*28
# then we wrapped it around the dataloader object in 'train_dataloader' and divided it by the batch of 64
# so we now have 60000/64 ~ 938 batches, each containing 64 images each of size 28*28
# so for the first batch we have 64 images and 1 image of size 28*28
# dimensions being -> (64,1,28,28)

print('length of training dataset', len(train_dataloader.dataset))
print('Number of training batches',int(len(train_dataloader)))
print()
print('From this we can conclude that when we say a dataloader,it has the dimension as follows')
print(f'dataloader has {len(train_dataloader)} batches, ')

print('length of testing dataset', len(test_dataloader.dataset))
print('Number of testing batches',int(len(test_dataloader)))

print()
for X,y in train_dataloader:
    print(f'train dataloader has {len(train_dataloader)} batches')
    print(f'Each batch has {X.shape[0]} images')
    print(f'Each image has dimension as {X.shape[2]}*{X.shape[3]}')
    print(f'Each batch has {y.shape} labels')
    print(y[0].item())
    break

length of training dataset 60000
Number of training batches 938

From this we can conclude that when we say a dataloader,it has the dimension as follows
dataloader has 938 batches, 
length of testing dataset 10000
Number of testing batches 157

train dataloader has 938 batches
Each batch has 64 images
Each image has dimension as 28*28
Each batch has torch.Size([64]) labels
9


In [5]:
# create a model
device = ('cpu')
print(f'Using {device} device')

Using cpu device


In [6]:
# define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
# loss function and an otpimizer 
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3)

In [8]:
def train(dataloader,model,loss_fun,optimizer):
    size = len(dataloader.dataset)
    
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        x,y = x.to(device),y.to(device)
        num_batches = len(x)
#         predict and calc error
        pred = model(x)
        loss = loss_fun(pred,y)
        
#         Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (batch%100 == 0) | (batch == len(dataloader)):
            loss, current = loss.item(),(batch+1)*num_batches
            print(f"Loss : {loss:>7f} [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader,model,loss_fun):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
#     torch.no_grad() is passed so that it does not calculate any gradients during testset and works with previously calculated ones
    with torch.no_grad():
        for x,y in dataloader:
            x,y = x.to(device),y.to(device)
            pred = model(x)
            test_loss += loss_fun(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error \n Accuracy : {(100*correct):>0.1f}% \n Avg_loss: {test_loss:>8f}\n")

In [10]:
epochs = 3
for epoch in range(epochs):
    print('epoch no : ',epoch+1)
    train(train_dataloader,model, loss_fun, optimizer)
    test(test_dataloader,model, loss_fun)
print('done')

epoch no :  1
Loss : 2.308845 [   64/60000]
Loss : 2.298676 [ 6464/60000]
Loss : 2.279666 [12864/60000]
Loss : 2.270586 [19264/60000]
Loss : 2.262078 [25664/60000]
Loss : 2.226506 [32064/60000]
Loss : 2.245336 [38464/60000]
Loss : 2.208933 [44864/60000]
Loss : 2.208906 [51264/60000]
Loss : 2.179075 [57664/60000]
Test Error 
 Accuracy : 31.1% 
 Avg_loss: 2.178170

epoch no :  2
Loss : 2.188558 [   64/60000]
Loss : 2.188727 [ 6464/60000]
Loss : 2.133457 [12864/60000]
Loss : 2.147375 [19264/60000]
Loss : 2.113538 [25664/60000]
Loss : 2.046915 [32064/60000]
Loss : 2.086796 [38464/60000]
Loss : 2.008084 [44864/60000]
Loss : 2.015861 [51264/60000]
Loss : 1.959991 [57664/60000]
Test Error 
 Accuracy : 55.7% 
 Avg_loss: 1.955016

epoch no :  3
Loss : 1.980700 [   64/60000]
Loss : 1.969141 [ 6464/60000]
Loss : 1.851595 [12864/60000]
Loss : 1.890064 [19264/60000]
Loss : 1.804870 [25664/60000]
Loss : 1.733507 [32064/60000]
Loss : 1.767676 [38464/60000]
Loss : 1.656680 [44864/60000]
Loss : 1.68376

In [11]:
torch.save(model.state_dict(),"model.pth")


In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>